In [1]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import os

os.environ["PYSPARK_PYTHON"]="/home/pc/g5_env_tf/bin/python39"

spark = SparkSession.builder.master("local[*]")\
            .appName("PosTagging")\
            .config('spark.executor.memory', '30g')\
            .config('spark.driver.maxResultSize', '30g')\
            .config('spark.driver.memory', '30g')\
            .getOrCreate()
            # .config('spark.ui.showConsoleProgress', False)\
           

sparkContext = spark.sparkContext

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 13:02:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/11 13:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/11 13:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/11 13:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/11 13:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
file_path = "hdfs://g5.bigtop.it:8020/user/root/language_social_media/part-00000-0fc8e338-2c87-4f9e-9db3-4476b0a96443-c000.snappy.parquet"

In [3]:
df1 = spark.read.option("header",True).parquet(file_path)
df1.show()

+--------------------+--------------------+-----------------+
|            sentence|       language_word|language_sentence|
+--------------------+--------------------+-----------------+
|selamat bungkusan...|  MS, MS, MS, MS, MS|               MS|
|poor product qual...|  EN, EN, EN, EN, EN|               EN|
|for product quali...| EN, EN, EN, OOV, EN|               EN|
|really good quali...|  EN, EN, EN, EN, EN|               EN|
|usual product qua...|  EN, EN, EN, EN, EN|               EN|
|and nice quality ...|  EN, EN, EN, EN, EN|               EN|
|money and quality...|  EN, EN, EN, EN, EN|               EN|
|glossy good quali...|  EN, EN, EN, EN, EN|               EN|
|selamat bungkusan...| MS, MS, MS, OOV, MS|               MS|
|好 是 卖家 还有 回复|  ZH, ZH, ZH, ZH, ZH|               ZH|
|yang cepat qualit...|  MS, MS, EN, MS, MS|               MS|
|dapat high qualit...|MS, EN, EN, OOV, OOV|               EN|
|money good qualit...|  EN, EN, EN, EN, EN|               EN|
|a proud quality

In [4]:
df1.rdd.getNumPartitions()

2

In [5]:
df1 = df1.repartition(110)

In [6]:
df1.rdd.getNumPartitions()

110

In [7]:
df1.show()

+--------------------+------------------+-----------------+
|            sentence|     language_word|language_sentence|
+--------------------+------------------+-----------------+
|muatttt jeee qual...| OOV, OOV, EN, OOV|               EN|
|the quality will ...|    EN, EN, EN, EN|               EN|
| 第一次 购买 也 并非|    ZH, ZH, ZH, ZH|               ZH|
|layanan seller qu...|MS, EN, EN, MS, MS|               MS|
|product quality a...|   EN, EN, OOV, EN|               EN|
|       多多关照 卖家|            ZH, ZH|               ZH|
|    quality gatal pd|      EN, OOV, OOV|               EN|
|     date 卖家 都 有|    EN, ZH, ZH, ZH|               ZH|
|money quality see...|    EN, EN, EN, EN|               EN|
|bad quality belt bru|    MS, EN, EN, MS|               EN|
|effectivenessukus...|    EN, MS, MS, MS|               MS|
|   high quality iove|        EN, EN, EN|               EN|
|quality dya berfu...|        EN, MS, MS|               MS|
|harga murah quali...|    MS, MS, EN, MS|               MS

In [8]:
from pyspark.sql.functions import col

language_none = df1.select(col("sentence"), col("language_word")).filter(df1.language_sentence == 'None')
language_en = df1.select(col("sentence"), col("language_word")).filter(df1.language_sentence == 'EN')
language_ms = df1.select(col("sentence"), col("language_word")).filter(df1.language_sentence == 'MS')
language_zh = df1.select(col("sentence"), col("language_word")).filter(df1.language_sentence == 'ZH')

In [9]:
language_none.show()

+-------------+-------------+
|     sentence|language_word|
+-------------+-------------+
|quality 거 거| EN, OOV, OOV|
|  quality ش ك| EN, OOV, OOV|
|quality 좋 아| EN, OOV, OOV|
|  quality ถ ู| EN, OOV, OOV|
|  quality ڤ ل| EN, OOV, OOV|
|quality 우 리| EN, OOV, OOV|
|quality 진 짜| EN, OOV, OOV|
|  quality ج ا| EN, OOV, OOV|
|quality 따 봉| EN, OOV, OOV|
|quality 짱 이| EN, OOV, OOV|
|quality 안 녕| EN, OOV, OOV|
|  quality θ θ| EN, OOV, OOV|
|quality ㅠ ㅠ| EN, OOV, OOV|
+-------------+-------------+



In [10]:
language_none.count()

13

In [11]:
language_en.show()

+--------------------+--------------------+
|            sentence|       language_word|
+--------------------+--------------------+
|quality delivery not|          EN, EN, EN|
|   quality next week|          EN, EN, EN|
|ordered quality w...|      EN, EN, EN, EN|
|good quality user...|      EN, EN, EN, EN|
|hati dengan quali...|  MS, MS, EN, EN, EN|
|      quality is out|          EN, EN, EN|
|        tuso quality|             OOV, EN|
| overall quality not|          EN, EN, EN|
|       fibre quality|              EN, EN|
|  quality amd faster|          EN, MS, EN|
|quality quality q...|  EN, EN, EN, EN, EN|
|pastu mang qualit...|OOV, MS, EN, OOV, MS|
|happy good qualit...| EN, EN, EN, OOV, EN|
|wear quality is soft|      EN, EN, EN, EN|
|products good qua...|     EN, EN, EN, OOV|
|buttt quality gooodd|          EN, EN, EN|
|     quality srs tak|         EN, OOV, MS|
|ok nice quality o...|  MS, EN, EN, MS, EN|
| bit slow quality ok|     OOV, EN, EN, MS|
|hope the quality ...| EN, EN, E

In [12]:
language_ms.show()

+--------------------+--------------------+
|            sentence|       language_word|
+--------------------+--------------------+
|    okey je kemas no|     EN, OOV, MS, EN|
|dibalut kemas sel...|     OOV, MS, EN, MS|
|packing cantik ke...|  EN, MS, MS, EN, MS|
| pun kemas bila saya|      MS, MS, MS, MS|
|nmpk kemas bile pkai|      MS, MS, EN, MS|
|memang kemas bole...|      MS, MS, MS, MS|
|baju nmpk kemas c...|  MS, MS, MS, MS, MS|
|quality dia berbaloi|          EN, MS, MS|
|cantik jahitan ke...|  MS, MS, MS, MS, MS|
|saya suka kemas c...|  MS, MS, MS, MS, MS|
|dan jahitan kemas...|      MS, MS, MS, MS|
| nmpk kemas and nmpk|      MS, MS, EN, MS|
|memang kemas soft...|      MS, MS, EN, EN|
|dan kemas sgt ber...|      MS, MS, EN, MS|
| pospaling pun kemas|         OOV, MS, MS|
|pembungkusan yg k...|MS, OOV, MS, OOV, MS|
|    masyaallah kemas|              MS, MS|
|       jshitan kemas|             OOV, MS|
|dan bungkus kemas...|  MS, MS, MS, MS, MS|
|amat kemas sekali...|          

In [13]:
language_ms.count()

202621

In [14]:
language_zh.show()

+--------------------+------------------+
|            sentence|     language_word|
+--------------------+------------------+
|      购买 大爱 产品|        ZH, ZH, ZH|
|  询问 了 卖家 他 跟|ZH, ZH, ZH, ZH, ZH|
|      到货 谢谢 卖家|        ZH, ZH, ZH|
|        所以 听 卖家|        ZH, ZH, ZH|
| 第一次 购买 可是 是|    ZH, ZH, ZH, ZH|
|  跟 这 卖家 下单 了|ZH, ZH, ZH, ZH, ZH|
|    第三次 购买 看到|        ZH, ZH, ZH|
|     再 回 购买 好评|    ZH, ZH, ZH, ZH|
|      一起 购买 孩子|        ZH, ZH, ZH|
|      卖家 服务 极差|        ZH, ZH, ZH|
|    第二次 购买 超级|        ZH, ZH, ZH|
|      希望 卖家 回复|        ZH, ZH, ZH|
|        good 卖家 的|        EN, ZH, ZH|
|哈哈哈 卖家 服务态度|        ZH, ZH, ZH|
| 垃圾 信息 卖家 卖家|    ZH, ZH, ZH, ZH|
|        优惠 卖家 还|        ZH, ZH, ZH|
|   拍照 卖家 客服 也|    ZH, ZH, ZH, ZH|
|   感谢 卖家 的 细心|    ZH, ZH, ZH, ZH|
|      口味 购买 两罐|        ZH, ZH, ZH|
|        卖家 才 发现|        ZH, ZH, ZH|
+--------------------+------------------+
only showing top 20 rows



In [15]:
from nltk import pos_tag, word_tokenize
def zh_pos_tag(text):    
    import logging
    import jieba
    import jieba.posseg
        
    jieba.setLogLevel(logging.WARNING)

    text = text.replace(' ','')
    results= jieba.posseg.lcut(text)
    empList = []
    for x, y in results:
        empList.append(y)
    
    return empList

In [16]:
def en_pos_tag(text):
    results = pos_tag(word_tokenize(text), lang='eng') 
    empList = []
    for x,y in results:
        empList.append(y)

    return empList

In [17]:
import malaya
import logging

def ms_pos_tag(text):
    # logging.basicConfig(level=logging.CRITICAL)
    results = malaya.pos.transformer('alxlnet', True).predict(text)
    
    empList = []

    for x, y in results:
        empList.append(y)
    return empList

2022-10-11 13:03:06.140826: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/pc/g5_env_tf/lib/python3.9/site-packages/malaya_boilerplate/frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
/home/pc/g5_env_tf/lib/python3.9/site-packages/malaya_boilerplate/frozen_graph.py:38: UserWarning: check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
  warnings.warn(
/home/pc/g5_env_tf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarni

In [18]:
def none_pos_tag(text):
    # logging.basicConfig(level=logging.CRITICAL)
    empList = []
    tempList = list(text.split(" "))
    for x in tempList:
        empList.append('XXX')
            
    return empList

In [19]:
language_en = language_en.repartition(110)

In [20]:
language_en.count()

283838

In [21]:
en_pos_tag = language_en.rdd.map(lambda x: (x[0], x[1], en_pos_tag(x[0]))).persist(StorageLevel.MEMORY_ONLY)

In [22]:
en_pos_tag.count()

283838

In [23]:
language_zh = language_zh.repartition(110)

In [24]:
language_zh.count()

47877

In [25]:
zh_pos_tag = language_zh.rdd.map(lambda x: (x[0], x[1], zh_pos_tag(x[0]))).persist(StorageLevel.MEMORY_ONLY)

In [26]:
zh_pos_tag.count()

47877

In [27]:
language_none= language_none.repartition(110)

In [28]:
language_none.count()

13

In [29]:
none_pos_tag = language_none.rdd.map(lambda x: (x[0], x[1], none_pos_tag(x[0]))).persist(StorageLevel.MEMORY_ONLY)

In [30]:
none_pos_tag.count()

13

### English, Chinese, None language Result

In [31]:
en_pos_tag.toDF().show()

+--------------------+--------------------+--------------------+
|                  _1|                  _2|                  _3|
+--------------------+--------------------+--------------------+
|quality delivery not|          EN, EN, EN|        [NN, NN, RB]|
|   quality next week|          EN, EN, EN|        [NN, IN, NN]|
|ordered quality w...|      EN, EN, EN, EN|   [VBN, NN, IN, DT]|
|good quality user...|      EN, EN, EN, EN|    [JJ, NN, NN, RB]|
|hati dengan quali...|  MS, MS, EN, EN, EN|[NN, JJ, NN, IN, NN]|
|      quality is out|          EN, EN, EN|       [NN, VBZ, RP]|
|        tuso quality|             OOV, EN|            [NN, NN]|
| overall quality not|          EN, EN, EN|        [JJ, NN, RB]|
|       fibre quality|              EN, EN|            [NN, NN]|
|  quality amd faster|          EN, MS, EN|       [NN, VBP, NN]|
|quality quality q...|  EN, EN, EN, EN, EN|[NN, NN, NN, JJ, NN]|
|pastu mang qualit...|OOV, MS, EN, OOV, MS|[NN, NN, NN, NN, NN]|
|happy good qualit...| EN

In [32]:
zh_pos_tag.toDF().show()

+--------------------+------------------+----------------+
|                  _1|                _2|              _3|
+--------------------+------------------+----------------+
|      购买 大爱 产品|        ZH, ZH, ZH|    [v, a, v, n]|
|  询问 了 卖家 他 跟|ZH, ZH, ZH, ZH, ZH|[v, ul, n, r, p]|
|      到货 谢谢 卖家|        ZH, ZH, ZH|      [v, nr, n]|
|        所以 听 卖家|        ZH, ZH, ZH|       [c, v, n]|
| 第一次 购买 可是 是|    ZH, ZH, ZH, ZH|    [m, v, c, v]|
|  跟 这 卖家 下单 了|ZH, ZH, ZH, ZH, ZH|[p, r, n, n, ul]|
|    第三次 购买 看到|        ZH, ZH, ZH|       [m, v, v]|
|     再 回 购买 好评|    ZH, ZH, ZH, ZH|    [d, v, v, v]|
|      一起 购买 孩子|        ZH, ZH, ZH|       [m, v, n]|
|      卖家 服务 极差|        ZH, ZH, ZH|      [n, vn, n]|
|    第二次 购买 超级|        ZH, ZH, ZH|       [m, v, b]|
|      希望 卖家 回复|        ZH, ZH, ZH|       [v, n, v]|
|        good 卖家 的|        EN, ZH, ZH|    [eng, n, uj]|
|哈哈哈 卖家 服务态度|        ZH, ZH, ZH|       [o, n, n]|
| 垃圾 信息 卖家 卖家|    ZH, ZH, ZH, ZH|    [n, n, n, n]|
|        优惠 卖家 还|        ZH, ZH, ZH

In [33]:
none_pos_tag.toDF().show(truncate = False)

+-------------+------------+---------------+
|_1           |_2          |_3             |
+-------------+------------+---------------+
|quality 거 거|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality ش ك  |EN, OOV, OOV|[XXX, XXX, XXX]|
|quality 좋 아|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality ถ ู  |EN, OOV, OOV|[XXX, XXX, XXX]|
|quality ڤ ل  |EN, OOV, OOV|[XXX, XXX, XXX]|
|quality 우 리|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality 진 짜|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality ج ا  |EN, OOV, OOV|[XXX, XXX, XXX]|
|quality 따 봉|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality 짱 이|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality 안 녕|EN, OOV, OOV|[XXX, XXX, XXX]|
|quality θ θ  |EN, OOV, OOV|[XXX, XXX, XXX]|
|quality ㅠ ㅠ|EN, OOV, OOV|[XXX, XXX, XXX]|
+-------------+------------+---------------+



In [34]:
zh_tags = {'a': 'ADJ',
 'ad': 'ACC_WORD',
 'ag': 'ADJ',
 'al': 'ADJ',
 'an': 'NOUN',
 'b': 'ADJ',
 'bl': 'ADJ',
 'c': 'CONJ',
 'cc': 'CONJ',
 'd': 'ADV',
 'f': 'NOUN',
 'm': 'NUM',
 'mq': 'MQ',
 'n': 'NOUN',
 'ng': 'XXX',
 'nl': 'NOUN',
 'nr': 'NOUN',
 'nr1': 'NOUN',
 'nr2': 'NOUN',
 'nrf': 'NOUN',
 'nrj': 'NOUN',
 'ns': 'NOUN',
 'nsf': 'NOUN',
 'nt': 'NOUN',
 'nz': 'NOUN',
 'p': 'IN',
 'pba': 'IN',
 'pbei': 'IN',
 'qt': 'QT',
 'qv': 'QV',
 'r': 'PRON',
 'rg': 'PRON',
 'rr': 'PRON',
 'ry': 'PRON',
 'rys': 'PRON',
 'ryt': 'PRON',
 'ryv': 'PRON',
 'rz': 'PRON',
 'rzs': 'PRON',
 'rzv': 'PRON',
 's': 'NOUN',
 't': 'TIME',
 'tg': 'TIME',
 'ul': 'UL',
 'v': 'VERB',
 'vd': 'ADX',
 'vf': 'VERB',
 'vg': 'VERB',
 'vi': 'VERB',
 'vl': 'VERB',
 'vn': 'NOUN_VERB',
 'vshi': 'VERB',
 'vx': 'VERB',
 'vyou': 'VERB',
 'x': 'XXX',
 'z': 'STATE_WORD'}

In [35]:
en_tags = {'CC': 'CCONJ',
 'CD': 'NOUN',
 'DT': 'DET',
 'EX': 'ADV',
 'FW': 'XXX',
 'IN': 'IN',
 'JJ': 'ADJ',
 'JJR': 'ADJ',
 'JJS': 'ADJ',
 'LS': 'XXX',
 'MD': 'VERB',
 'NN': 'NOUN',
 'NNP': 'PROPN',
 'NNPS': 'PROPN',
 'NNS': 'NOUN',
 'PDT': 'DET',
 'POS': 'POS',
 'PRP': 'PRON',
 'PRP$': 'PRON',
 'RB': 'ADV',
 'RBR': 'ADV',
 'RBS': 'ADV',
 'RP': 'PART',
 'TO': 'TO',
 'UH': 'UH',
 'VB': 'VERB',
 'VBD': 'VERB',
 'VBG': 'VERB',
 'VBN': 'VERB',
 'VBP': 'VERB',
 'VBZ': 'VERB',
 'WDT': 'SCONJ',
 'WP': 'SCONJ',
 'WP$': 'SCONJ',
 'WRB': 'SCONJ'}

In [36]:
def standardizer(output_list, dictionary):
    empList = []
    
    for x in output_list:
        if x not in dictionary:
            empList.append('XXX')
            continue
        y = dictionary.get(x)
        empList.append(y)
    return empList

In [37]:
standard_zh_pos_tag = zh_pos_tag.map(lambda x: (x[0], x[1], standardizer(x[2], zh_tags))).persist(StorageLevel.MEMORY_ONLY)

In [38]:
standard_zh_pos_tag.count()

47877

In [39]:
standard_zh_pos_tag.toDF().show()

+--------------------+------------------+--------------------+
|                  _1|                _2|                  _3|
+--------------------+------------------+--------------------+
|      购买 大爱 产品|        ZH, ZH, ZH|[VERB, ADJ, VERB,...|
|  询问 了 卖家 他 跟|ZH, ZH, ZH, ZH, ZH|[VERB, UL, NOUN, ...|
|      到货 谢谢 卖家|        ZH, ZH, ZH|  [VERB, NOUN, NOUN]|
|        所以 听 卖家|        ZH, ZH, ZH|  [CONJ, VERB, NOUN]|
| 第一次 购买 可是 是|    ZH, ZH, ZH, ZH|[NUM, VERB, CONJ,...|
|  跟 这 卖家 下单 了|ZH, ZH, ZH, ZH, ZH|[IN, PRON, NOUN, ...|
|    第三次 购买 看到|        ZH, ZH, ZH|   [NUM, VERB, VERB]|
|     再 回 购买 好评|    ZH, ZH, ZH, ZH|[ADV, VERB, VERB,...|
|      一起 购买 孩子|        ZH, ZH, ZH|   [NUM, VERB, NOUN]|
|      卖家 服务 极差|        ZH, ZH, ZH|[NOUN, NOUN_VERB,...|
|    第二次 购买 超级|        ZH, ZH, ZH|    [NUM, VERB, ADJ]|
|      希望 卖家 回复|        ZH, ZH, ZH|  [VERB, NOUN, VERB]|
|        good 卖家 的|        EN, ZH, ZH|    [XXX, NOUN, XXX]|
|哈哈哈 卖家 服务态度|        ZH, ZH, ZH|   [XXX, NOUN, NOUN]|
| 垃圾 信息 卖家 卖家|    

In [40]:
standard_en_pos_tag = en_pos_tag.map(lambda x: (x[0], x[1], standardizer(x[2], en_tags))).persist(StorageLevel.MEMORY_ONLY)

In [41]:
standard_en_pos_tag.count()

283838

In [42]:
standard_en_pos_tag.toDF().show()

+--------------------+--------------------+--------------------+
|                  _1|                  _2|                  _3|
+--------------------+--------------------+--------------------+
|quality delivery not|          EN, EN, EN|   [NOUN, NOUN, ADV]|
|   quality next week|          EN, EN, EN|    [NOUN, IN, NOUN]|
|ordered quality w...|      EN, EN, EN, EN|[VERB, NOUN, IN, ...|
|good quality user...|      EN, EN, EN, EN|[ADJ, NOUN, NOUN,...|
|hati dengan quali...|  MS, MS, EN, EN, EN|[NOUN, ADJ, NOUN,...|
|      quality is out|          EN, EN, EN|  [NOUN, VERB, PART]|
|        tuso quality|             OOV, EN|        [NOUN, NOUN]|
| overall quality not|          EN, EN, EN|    [ADJ, NOUN, ADV]|
|       fibre quality|              EN, EN|        [NOUN, NOUN]|
|  quality amd faster|          EN, MS, EN|  [NOUN, VERB, NOUN]|
|quality quality q...|  EN, EN, EN, EN, EN|[NOUN, NOUN, NOUN...|
|pastu mang qualit...|OOV, MS, EN, OOV, MS|[NOUN, NOUN, NOUN...|
|happy good qualit...| EN

### Malay Pos Tagging

In [43]:
language_ms= language_ms.repartition(110)

In [44]:
language_ms.count()

202621

In [45]:
sampling_ms = language_ms.limit(10000)

In [46]:
sampling_ms.count()

10000

In [47]:
sampling_ms = sampling_ms.repartition(220)

In [48]:
sampling_ms.count()

10000

In [49]:
ms_pos_tag = sampling_ms.rdd.map(lambda x: (x[0], x[1], ms_pos_tag(x[0]))).persist(StorageLevel.MEMORY_ONLY)

In [50]:
ms_pos_tag.toDF().show()

2022-10-11 13:03:25.137784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/pc/g5_env_tf/lib/python3.9/site-packages/malaya_boilerplate/frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
/home/pc/g5_env_tf/lib/python3.9/site-packages/malaya_boilerplate/frozen_graph.py:38: UserWarning: check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
  warnings.warn(
/home/pc/g5_env_tf/lib/python3.9/site-packages/malaya/tokenizer.py:202: F

+--------------------+--------------------+--------------------+
|                  _1|                  _2|                  _3|
+--------------------+--------------------+--------------------+
|           dan kemas|              MS, MS|       [CCONJ, NOUN]|
|smpaibalut dgn ke...|OOV, EN, MS, MS, OOV|[VERB, ADP, NOUN,...|
|     kemas wangi sgt|          MS, MS, EN|   [NOUN, NOUN, ADJ]|
|orderbox cntikpac...|      EN, MS, MS, MS|[NOUN, NOUN, PROP...|
|quality superb wa...|         EN, OOV, MS| [NOUN, NOUN, SCONJ]|
|          lock kemas|              EN, MS|         [NOUN, ADJ]|
|tepi kurang kemas...|  MS, MS, MS, MS, MS|[NOUN, ADV, PROPN...|
|      and pack kemas|          EN, EN, MS| [CCONJ, NOUN, NOUN]|
|dibungkus dgn kem...|      MS, EN, MS, MS|[VERB, ADP, NOUN,...|
| quality saiz memang|         EN, OOV, MS|  [PROPN, NOUN, ADV]|
|buku elok kemas b...| MS, MS, MS, OOV, EN|[NOUN, ADJ, ADJ, ...|
|    kemas helok tqsm|         MS, OOV, MS|[NOUN, PROPN, PROPN]|
|dn packing kemas ...| OO

In [51]:
%%time
ms_pos_tag.count()

2022-10-11 13:09:22.812899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-11 13:09:22.826022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-11 13:09:22.828592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

KeyboardInterrupt: 

In [52]:
column = ['sentence', 'language', 'pos_tag']
df_zh = standard_zh_pos_tag.toDF(column)
df_en = standard_en_pos_tag.toDF(column)
df_ms = ms_pos_tag.toDF(column)
df_none = none_pos_tag.toDF(column)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/pc/g5_env_tf/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/pc/g5_env_tf/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/python39/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_final = df_zh.union(df_en).union(df_ms).union(df_none).persist(StorageLevel.MEMORY_ONLY)

In [ ]:
df_final.show()

In [ ]:
from pyspark.sql.functions import col, concat_ws
df_final = df_final.withColumn('pos_tag', concat_ws(', ', col('pos_tag')))

In [ ]:
df_final.show()

In [ ]:
import pyspark.sql.functions as f
df_final = df_final.withColumn('n-gram', f.size(f.split(f.col('pos_tag'), ' ')))

In [ ]:
df_final = df_final.filter(col('n-gram') < 6)

In [ ]:
df_final.show()